This package is to run data modelling/forecast predictions based on historical data.

The sqlite database keeps a record of what stock data is on hand, when it begins, and when it ends.

When data is requested from the database, if it is not available in the database, execute a request to the API (and update the ticker table as well as the metadata table in the database)

In [1]:
import datetime as dt
from zoneinfo import ZoneInfo
import yfinance as yf
import sqlite3
import pandas as pd
import time 
from db_utilities import query, list_all_tables, create_metadata_table, create_ticker_table, drop_ticker_table, read_table
from stock_functions import get_close_price, get_ohlc
# populate your universe of stocks
data = pd.read_csv('data/constituents.csv')
all_tickers = list(data['Symbol'])

In [2]:
aapl_1 = get_ohlc('AAPL', start_date='2016-01-01', end_date='2018-01-01')

c:\Users\polan\Desktop\GitHub Repos\A New Project\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [3]:
aapl_2 = get_ohlc('AAPL', start_date='2018-01-01')

c:\Users\polan\Desktop\GitHub Repos\A New Project\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [4]:
con = sqlite3.connect('stocks.db')

list(query("SELECT * FROM stock_metadata WHERE stock_metadata.ticker = 'AAPL';").values)

[]

In [5]:
def get_metadata(table_name, METADATA_TABLE='stock_metadata'):

    try:
        output = list(query(f"SELECT * FROM {METADATA_TABLE} WHERE {METADATA_TABLE}.ticker = '{table_name}';").values)
        # print(f'drew {len(output)} from {METADATA_TABLE}')
    except BaseException as e:
        print(e)
        return None
    return output

In [6]:
get_metadata('AAPL')


[]

In [8]:
def get_existing_metadata(table_name, METADATA_TABLE='stock_metadata'):

    try:
        output = list(query(f"SELECT * FROM {METADATA_TABLE} WHERE {METADATA_TABLE}.ticker = '{table_name}';").values)
        # print(f'drew {len(output)} from {METADATA_TABLE}')
    except BaseException as e:
        print(e)
        return None
    return output

In [11]:
def tz_aware_first_and_last(table_name, timezone='America/New_York'):
    return [dt.datetime(*tuple([int(y) for y in x[:10].split('-')])).replace(tzinfo=ZoneInfo(timezone)) for x in query(f"SELECT max(date) as most_recent, min(date) as oldest FROM {table_name};").values[0]]

In [15]:
tz_aware_first_and_last('AAPL')

TypeError: 'NoneType' object is not subscriptable

In [16]:
def update_ticker_table(table_name:str, 
                        start_date:str,
                        end_date:str,
                        db_name='stocks.db', METADATA_TABLE:str='stock_metadata'):

    try:
        last_date, first_date, = tz_aware_first_and_last(table_name)
    except:
        last_date = 0
        first_date = 0
    data = get_ohlc(ticker_name=table_name, start_date=start_date, end_date=end_date)
    if data['Date'].min() > last_date:   
    con = sqlite3.connect(db_name)

    columns = ",".join([f"'{x}'" for x in data.columns])
    with con:
        try:
            for row in data.iterrows():
                values = ",".join([f"'{x}'" for x in row[1].values])
                #print(f'INSERT INTO {table_name} ({columns}) VALUES ({values})')
                con.execute(f'INSERT INTO {table_name} ({columns}) VALUES ({values})')
        except:
            con.rollback()


    new_first_date = min([data['Date'].min(), first_date])
    new_last_date = max([data['Date'].max(), last_date])  
    print(new_first_date, new_last_date)
    update_metadata(table_name, str(new_first_date)[:10], str(new_last_date)[:10], )

